In [ ]:
# Semantic Segmentation Demo

This notebook demonstrates the semantic segmentation capabilities of our project. We support multiple models:
1. SegFormer-B0 (from Hugging Face)
2. DeepLabV3+ (with Edge TPU optimization)
3. Mosaic (with Cityscapes classes)

Each model can segment images into different semantic classes like roads, buildings, people, cars, etc.


In [ ]:
# Import required libraries
import os
import sys
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Add the project root to Python path
SCRIPT_DIR = os.path.dirname(os.path.abspath('.'))


In [ ]:
# Import project modules
from eah_segmentation.inference import run_inference_on_image
from eah_segmentation.visualization import colorize_mask

# Print TensorFlow version and GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
# Helper function to display results
def display_results(image, prediction, title="Segmentation Results"):
    """Display the original image, segmentation mask, and overlay."""
    plt.figure(figsize=(15, 5))
    
    # Original image
    plt.subplot(131)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')
    
    # Segmentation mask
    plt.subplot(132)
    colored_mask = colorize_mask(prediction)
    plt.imshow(colored_mask)
    plt.title("Segmentation Mask")
    plt.axis('off')
    
    # Overlay
    plt.subplot(133)
    overlay = cv2.addWeighted(image, 0.7, colored_mask, 0.3, 0)
    plt.imshow(overlay)
    plt.title("Overlay")
    plt.axis('off')
    
    plt.suptitle(title)
    plt.show()


In [ ]:
# Load and preprocess an image
def load_image(image_path, target_size=(512, 512)):
    """Load and preprocess an image for segmentation."""
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image at {image_path}")
    
    # Convert BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize image
    img = cv2.resize(img, target_size)
    
    # Normalize to [0, 1]
    img = img.astype(np.float32) / 255.0
    
    return img


In [ ]:
# Load models
def load_model(model_name):
    """Load a segmentation model."""
    model_dir = os.path.join('models', model_name)
    if not os.path.exists(model_dir):
        raise ValueError(f"Model directory not found: {model_dir}")
    
    model = tf.saved_model.load(model_dir)
    print(f"Successfully loaded {model_name} model")
    return model

# Available models
MODELS = {
    'segformer_b0': {
        'input_size': (512, 512),
        'description': 'Lightweight SegFormer model for general scene segmentation'
    },
    'deeplabv3plus_edgetpu': {
        'input_size': (512, 512),
        'description': 'Edge TPU optimized DeepLabV3+ for efficient inference'
    }
}


In [ ]:
# Running Inference

Now let's try running inference with our models. First, we'll need to:
1. Load a test image
2. Load the models
3. Run inference and visualize results

Let's start with loading a test image from the data directory.


In [ ]:
# Load a test image
# You can replace this with any image path
image_path = os.path.join('data', 'test_image.jpg')  # Update this path
try:
    image = load_image(image_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.title("Test Image")
    plt.axis('off')
    plt.show()
except Exception as e:
    print(f"Error loading image: {str(e)}")
    print("Please make sure to place a test image in the data directory")


In [ ]:
# Run inference with each model
for model_name, model_info in MODELS.items():
    print(f"\nRunning inference with {model_name}...")
    try:
        # Load model
        model = load_model(model_name)
        
        # Run inference
        predictions = run_inference_on_image(model, image, model_name)
        
        # Display results
        display_results(image, predictions, f"Results from {model_name}")
        
        # Print unique classes found
        unique_classes = np.unique(predictions)
        print(f"\nDetected classes in {model_name}:")
        for cls in unique_classes:
            num_pixels = np.sum(predictions == cls)
            percentage = (num_pixels / predictions.size) * 100
            print(f"Class {cls}: {num_pixels} pixels ({percentage:.1f}%)")
            
    except Exception as e:
        print(f"Error running {model_name}: {str(e)}")


In [ ]:
# Presentation Demo

This notebook demonstrates the semantic segmentation capabilities.


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
